# Results of LeNet Trained on MNIST

In this notebook I display results in terms of accuracy on test data for LeNet models trained on the MNIST dataset using various regularization techniques. The LeNet model and the MNIST dataset were set up as in Hoffman 2019 (for all the models). The MNIST data is preprocessed by normalizing using mean 0.1307 and variance 0.3081. The batch size is 100. The model optimizes using SGD with momentum p = 0.9, and standard cross-entropy loss. Model parameters are initialized using Glorot initialization (See Glorot & Bengio 2010), expect for SVB regularization which uses orthogonal initialization. Models are trained with no regularization, L2 regularization, SVB regularization and Jacobian regularization, both with and without dropout with a dropout rate of p_drop = 0.5. The L2 regularization coefficient and Jacobian regularization coefficient are the same as in Hoffman 2019: l2_lmbd = 0.0005 and lambda_jacobian_reg = 0.01. For SVB regularization I use the hyperparameters svb_freq=600 and svb_eps = 0.05. The learning rate starts at 0.1, and is reduced to 0.01 and 0.001 1/3 and 2/3s into training, respectively. The models are trained for 250 epochs.

### Imports and Model Loading

In [1]:
import jupyter_black
import numpy as np
import pandas as pd
import torch
from scipy import stats
from torchsummary import summary
from tqdm import tqdm

from data_generators import data_loader_MNIST
from model_classes import LeNet_MNIST
from plotting_tools import get_random_img, generate_random_vectors
from tools import accuracy, compute_total_variation, ModelInfo

jupyter_black.load()

In [2]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load MNIST data
train_loader, test_loader = data_loader_MNIST()

# Summary of model
summary_model = LeNet_MNIST().to(device)
summary(summary_model, (1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]             156
         MaxPool2d-2            [-1, 6, 14, 14]               0
            Conv2d-3           [-1, 16, 10, 10]           2,416
         MaxPool2d-4             [-1, 16, 5, 5]               0
            Linear-5                  [-1, 120]          48,120
           Dropout-6                  [-1, 120]               0
            Linear-7                   [-1, 84]          10,164
           Dropout-8                   [-1, 84]               0
            Linear-9                   [-1, 10]             850
Total params: 61,706
Trainable params: 61,706
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.06
Params size (MB): 0.24
Estimated Total Size (MB): 0.30
---------------------------------------------

In [3]:
# Load models
dataset = "mnist"

# Defining model types and versions
model_types = ["no_reg", "l2", "jacobi", "svb"]
model_versions = ["", "_no_dropout"]

model_names = []

for t in model_types:
    for v in model_versions:
        for i in range(5):
            model_names.append(f"model_{t}{v}_{i}")

models = {name: ModelInfo(name, dataset) for name in model_names}

### Accuracies

Here I calculate the accuracies of each model as a 95% confidence interval. I have trained five models of each variation that I use to calculate the CI.

In [4]:
# Dictionary to hold accuracy data
accuracy_data = {}

# Calculate accuracies with 95% CI for models
for t in model_types:
    for v in model_versions:
        accuracies = []
        for i in range(5):  # Assuming 5 versions of each model
            model_name = f"model_{t}{v}_{i}"
            model = models[model_name].model
            acc = accuracy(model, test_loader, device)
            accuracies.append(acc)

        # Calculate mean accuracy
        mean_accuracy = np.mean(accuracies)

        # Calculate standard error
        std_error = stats.sem(accuracies)

        # Calculate confidence interval
        CI = std_error * stats.t.ppf((1 + 0.95) / 2, len(accuracies) - 1)

        # Store results in the data dictionary
        accuracy_data[f"{t}{v if v != '' else '_dropout'}"] = (mean_accuracy, CI)

In [5]:
# Convert the dictionary into a DataFrame
df = pd.DataFrame.from_dict(
    accuracy_data, orient="index", columns=["Mean Accuracy", "Confidence Interval"]
)

# Convert the numbers from decimal to percentage and round to four decimal places
df = df.multiply(100).round(3)

# Create a new column that combines Mean Accuracy and Confidence Interval
df["Accuracy +/- CI (95%)"] = df.apply(
    lambda row: f"{row['Mean Accuracy']} +/- {row['Confidence Interval']}", axis=1
)

# Drop the original columns
df = df.drop(columns=["Mean Accuracy", "Confidence Interval"])

# Reset the index
df = df.reset_index().rename(columns={"index": "Model"})

display(df)

,Model,Accuracy +/- CI (%)
0,no_regdropout,98.684 +/- 0.074
1,no_reg_no_dropout,98.872 +/- 0.088
2,l2dropout,99.144 +/- 0.087
3,l2_no_dropout,99.162 +/- 0.059
4,jacobidropout,98.652 +/- 0.111
5,jacobi_no_dropout,98.846 +/- 0.087
6,svbdropout,98.416 +/- 0.082
7,svb_no_dropout,98.716 +/- 0.052


### Total Variation

Total variation is the sum of the absolute differences for neighboring pixel-values in the input images. This measures how much noise is in the images. I generate 25 different images for each model to get a good mean and standard deviation, and give the results as a tabel for each model with the eight models (models with and without dropout) as rows, and the three different zoom levels as columns. The tabel contains the mean and 95 % confidence interval for the total variation for each model at each zoom level.

In [6]:
# Define zoom levels, number of images, and confidence level for confidence interval calculation
zoom_levels = [0.025, 0.01, 0.001]
n_images = 5
confidence_level = 0.95

# Select only the first model of each type (those with index 0)
selected_models = [name for name in model_names if name.split("_")[-1] == "0"]

# Dataframe to store results
cols = pd.MultiIndex.from_product([zoom_levels, ["mean", "conf_interval"]])
df_results = pd.DataFrame(index=selected_models, columns=cols)

# Loop over the selected models
for name in tqdm(selected_models):
    model = models[name].model  # Get model
    model.to(device)
    tv_values = {
        zoom: [] for zoom in zoom_levels
    }  # To store total variation values for each zoom level

    # Generate tv values for n_images number of images
    for _ in range(n_images):
        # Get random image and vectors
        img = get_random_img(test_loader)
        v1, v2 = generate_random_vectors(img)

        # Compute the total variation of decision boundaries for the current model
        # and the generated image at different zoom levels
        tv_list = compute_total_variation(model, img, v1, v2, device)
        for zoom, tv in zip(zoom_levels, tv_list):
            tv_values[zoom].append(tv)

    # Calculate mean and 95% confidence interval for total variation values at each zoom level
    for zoom in zoom_levels:
        mean = np.mean(tv_values[zoom])
        std_err = np.std(tv_values[zoom]) / np.sqrt(n_images)
        conf_interval = stats.t.ppf((1 + confidence_level) / 2, n_images - 1) * std_err
        df_results.loc[name, (zoom, "mean")] = mean
        df_results.loc[name, (zoom, "conf_interval")] = conf_interval

  0%|          | 0/8 [00:00<?, ?it/s]

c:\ProgramData\Anaconda3\envs\bioai\Lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ..\aten\src\ATen\native\TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
100%|██████████| 8/8 [09:21<00:00, 70.15s/it]


In [7]:
display(df_results)

0.025                  0.010                \
                              mean conf_interval     mean conf_interval   
model_no_reg_0             11136.0  12087.528961  32418.8  14361.802687   
model_no_reg_no_dropout_0   3153.4   2817.265248  14891.6   2396.000587   
model_l2_0                  2587.8   3105.427863   3912.2   1422.556057   
model_l2_no_dropout_0       2314.4   1343.378447   4306.0   1421.014658   
model_jacobi_0             12300.8  16727.691032  22507.0  14052.495028   
model_jacobi_no_dropout_0   6428.4   2015.106843   5586.8   4873.296578   
model_svb_0                 1894.2   2198.133792   3847.8   3453.741244   
model_svb_no_dropout_0       250.8    622.818661   3314.0   1675.660782   

                             0.001                
                              mean conf_interval  
model_no_reg_0             38731.2   9697.810269  
model_no_reg_no_dropout_0  38129.8  16486.176039  
model_l2_0                  9070.0   2654.638949  
model_l2_no_dropout_0      11232.0   7377.217718  
model_jacobi_0             11696.2   3344.269092  
model_jacobi_no_dropout_0   8910.2   9543.666266  
model_svb_0                 2254.2   2083.163695  
model_svb_no_dropout_0     11759.2   2865.032568